####            <center>** Little Lemon analysis and sales report ** </center>

## Task#1

In [1]:
from mysql.connector.pooling import MySQLConnectionPool
import mysql.connector as connector

dbconfig = {
    "database":"little_lemon_db",
    "user" : "root",
    "password" : ""
}

try:
    pool = MySQLConnectionPool(
                        pool_name='pool_b',
                        pool_size=2,
                        **dbconfig)
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except connector.Error as err:
    print(f"Error Message : {err.msg}")
    print(f"Error No : {err.errno}")    

The connection pool is created with a name:  pool_b
The pool size is: 2


## Task#2

In [2]:
# Get the connection from the connection pool "pool"
print("Getting a connection from the pool.")
connection1 = pool.get_connection()
print("'connection1' object is created with a connection from the pool")
insert_bookings="""
                INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName,
                BookingSlot, EmployeeID)
                VALUES
                (8, 'Anees','Java','18:00:00',6)
                """
cursor = connection1.cursor()
cursor.execute("USE little_lemon_db")
cursor.execute(insert_bookings)      
connection1.commit()      

Getting a connection from the pool.
'connection1' object is created with a connection from the pool


In [3]:
# Get the connection from the connection pool "pool"
print("Getting a connection from the pool.")
connection2 = pool.get_connection()
print("'connection2' object is created with a connection from the pool")
insert_bookings="""
                INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName,
                BookingSlot, EmployeeID)
                VALUES
                (5, 'Bald','Vin','19:00:00',6)
                """
cursor = connection2.cursor()
cursor.execute("USE little_lemon_db")
cursor.execute(insert_bookings)      
connection2.commit()                  


Getting a connection from the pool.
'connection2' object is created with a connection from the pool


In [4]:
# Get the connection from the connection pool "pool"
try:        
    print("Getting a connection from the pool.")
    connection3 = pool.get_connection()
    print("'connection3' object is created with a connection from the pool")         
    
except connector.Error as err:
    print(f"Connection Pool ERROR : {err.msg}")
    print("No More Connections Are Available. \n")
    print("Adding New Connection in the pool") 
    connection = connector.connect(user="root",password="")
    pool.add_connection(cnx=connection)
    print("A new connection is added in the pool.\n")    
    print("Getting a connection from the pool.")
    connection3 = pool.get_connection()
    print("'connection3' object is created with a connection from the pool")
    
    
insert_bookings="""
                INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName,
                BookingSlot, EmployeeID)
                VALUES
                (12, 'Jay','Kon','19:30:00',6)
                """
cursor = connection3.cursor()
cursor.execute("USE little_lemon_db")
cursor.execute(insert_bookings)      
connection3.commit()                 


Getting a connection from the pool.
Connection Pool ERROR : Failed getting connection; pool exhausted
No More Connections Are Available. 

Adding New Connection in the pool
A new connection is added in the pool.

Getting a connection from the pool.
'connection3' object is created with a connection from the pool


In [5]:
for connection in [connection1,connection2, connection3]:
    try:
        connection.close()
        print("Connection is returned to the pool")
    except connector.Error as err:
        print("\nConnection can't be returned to the pool")
        print("Error message:", err.msg)


Connection is returned to the pool
Connection is returned to the pool

Connection can't be returned to the pool
Error message: Failed adding connection; queue is full


## Task#3

In [6]:
select_employee_manager = """
SELECT EmployeeID,Name FROM Employees
WHERE Role = 'Manager'; 
"""

select_employee_highest_Salary = """
SELECT Name,Role FROM Employees
ORDER BY Annual_Salary DESC
LIMIT 1;
"""

select_count_booking_slot = """
SELECT COUNT(BookingID) AS 'Number of Guests' FROM Bookings
WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';
"""

select_guests_info = """
SELECT BookingID, CONCAT(GuestFirstName,' ',GuestLastName) AS GuestFullName
FROM Bookings
ORDER BY BookingSlot ASC;
"""

In [7]:
#The name and EmployeeID of the Little Lemon manager.
cursor.execute(select_employee_manager)
result = cursor.fetchall()
cols = cursor.column_names
print("The name and EmployeeID of the Little Lemon manager : \n")
print(cols)
for res in result:
    print(res)

The name and EmployeeID of the Little Lemon manager : 

('EmployeeID', 'Name')
(1, 'Mario Gollini')


In [8]:
#The name and role of the employee who receives the highest salary.
cursor.execute(select_employee_highest_Salary)
result = cursor.fetchall()
cols = cursor.column_names
print("The name and role of the employee who receives the highest salary : \n")
print(cols)
for res in result:
    print(res)

The name and role of the employee who receives the highest salary : 

('Name', 'Role')
('Mario Gollini', 'Manager')


In [9]:
#The number of guests booked between 18:00 and 20:00.
cursor.execute(select_count_booking_slot)
result = cursor.fetchall()
cols = cursor.column_names
print("The number of guests booked between 18:00 and 20:00 : \n")
print(cols)
for res in result:
    print(res)

The number of guests booked between 18:00 and 20:00 : 

('Number of Guests',)
(7,)


In [10]:
# The full name and BookingID of all guests waiting 
# to be seated with the receptionist in sorted order with respect to their BookingSlot.
cursor.execute(select_guests_info)
result = cursor.fetchall()
cols = cursor.column_names
print("The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot : \n")
print(cols)
for res in result:
    print(res)

The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot : 

('BookingID', 'GuestFullName')
(3, 'Vanessa McCarthy')
(4, 'Marcos Romero')
(7, 'Anees Java')
(5, 'Hiroki Yamane')
(1, 'Anna Iversen')
(2, 'Joakim Iversen')
(8, 'Bald Vin')
(9, 'Jay Kon')
(6, 'Diana Pinto')


## Task#4

In [11]:
cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport")

In [12]:
select_stats = """
CREATE PROCEDURE IF NOT EXISTS BasicSalesReport()
BEGIN
SELECT 
SUM(BillAmount) AS 'Total Sales',
AVG(BillAmount) AS 'Average sale',
MIN(BillAmount) AS 'Minimum bill paid',
MAX(BillAmount) AS 'Maximum bill paid'
FROM Orders;
END
"""

In [13]:
cursor.execute(select_stats)
cursor.callproc("BasicSalesReport")
results = next(cursor.stored_results())
dataset = results.fetchall()
cols = results.column_names
print("Sales Statistics : \n")
print(cols)
for data in dataset:
    print(data)

Sales Statistics : 

('Total Sales', 'Average sale', 'Minimum bill paid', 'Maximum bill paid')
(Decimal('243'), Decimal('48.6000'), 37, 86)


In [14]:
for column_id in cursor.stored_results():
    cols = [column[0] for column in column_id.description]

    
print("Today's sales report:")
for result in dataset:
    print("\t",cols[0],":",result[0])
    print("\t",cols[1],":",result[1])
    print("\t",cols[2],":",result[2])
    print("\t",cols[3],":",result[3])

Today's sales report:
	 Total Sales : 243
	 Average sale : 48.6000
	 Minimum bill paid : 37
	 Maximum bill paid : 86


## Task#5

In [23]:
connection = pool.get_connection()

cursor = connection.cursor(buffered= True)
cursor.execute("USE little_lemon_db")

query = """
SELECT 
Bookings.BookingSlot,
CONCAT(Bookings.GuestFirstName," ",Bookings.GuestLastName) AS Guest_Name,
Employees.Name AS Employee_Name,
Employees.Role AS Employee_Role
FROM Bookings
INNER JOIN Employees ON Bookings.EmployeeID = Employees.EmployeeID
ORDER BY Bookings.BookingSlot ASC;
"""

cursor.execute(query)

results=cursor.fetchmany(size=3)


cols = cursor.column_names
for result in results:
    print(f"\n Booking Slot: {result[0]}")
    print(f"\tGuest Name: {result[1]} ")
    print(f"\t Assigned To: {result[2]}  [{result[3]}]")  
    
try:
    connection.close()
    print("Connection is returned to the pool")
except connector.Error as err:
    print("\nConnection can't be returned to the pool")
    print("Error message:", err.msg)    

('BookingSlot', 'Guest_Name', 'Employee_Name', 'Employee_Role')

 Booking Slot: 15:00:00
	Guest Name: Vanessa McCarthy 
	 Assigned To: Giorgos Dioudis  [Head Chef]

 Booking Slot: 17:30:00
	Guest Name: Marcos Romero 
	 Assigned To: Fatma Kaya  [Assistant Chef]

 Booking Slot: 18:00:00
	Guest Name: Anees Java 
	 Assigned To: John Millar  [Receptionist]
Connection is returned to the pool
